In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pypsa

plt.style.use(["bmh", "../../matplotlibrc"])
sns.set_palette("colorblind")

import matplotlib as mpl

mpl.rcParams["figure.dpi"] = 100

import yaml

with open("../../config/config.yaml", "r") as file:
    config = yaml.safe_load(file)

In [ ]:
voll = pypsa.Network(
    "../../results/20240506-num-years/network_solved_LT-country+DE-number_years+70-voll+true.nc"
)
pwl = pypsa.Network(
    "../../results/20240506-num-years/network_solved_LT-country+DE-number_years+70-elastic_pwl+default.nc"
)

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(10, 6))

price = pwl.buses_t.marginal_price["electricity"].resample("YE").mean()
price.plot(
    ax=ax[0],
    color=config["colors"]["hydrogen"],
    drawstyle="steps-post",
    label="PWL-elastic",
)
ax[0].axhline(price.mean(), color="#898989", linewidth=1, label="mean (PWL-elastic)")
price = voll.buses_t.marginal_price["electricity"].resample("YE").mean()
price.plot(
    ax=ax[0],
    color=config["colors"]["hydrogen"],
    linestyle=":",
    drawstyle="steps-post",
    label="VOLL",
)
ax[0].axhline(
    price.mean(), color="#898989", linewidth=1, label="mean (VOLL)", linestyle=":"
)
ax[0].legend(ncol=2)

msv = pwl.buses_t.marginal_price["hydrogen"]
msv.plot(ax=ax[1], color=config["colors"]["hydrogen"], label="PWL-elastic")
ax[1].axhline(msv.mean(), color="#898989", linewidth=1, label="mean (PWL-elastic)")
msv = voll.buses_t.marginal_price["hydrogen"]
msv.plot(ax=ax[1], color=config["colors"]["hydrogen"], label="VOLL", linestyle=":")
ax[1].axhline(
    msv.mean(), color="#898989", linewidth=1, label="mean (VOLL)", linestyle=":"
)
ax[1].legend(ncol=2)

cf = pwl.generators_t.p_max_pu.loc[:, ::-1]
deviation = (cf.resample("YE").mean() - cf.mean()) / cf.mean() * 100

deviation.plot(ax=ax[2], drawstyle="steps-post", color=config["colors"])
ax[0].set_ylabel("Average Annual\nElectricity Price\n[€/MWh]")
ax[0].set_xlabel("")
ax[1].set_ylabel("Hourly Hydrogen\nMarginal Storage\nValue [€/MWh]")
ax[1].set_xlabel("")
ax[2].set_ylabel("Average Annual\nCapacity Factor\nAnomaly [%]")
ax[2].set_xlabel("")
ax[2].set_ylim(-20, 20)
ax[2].axhline(0, color="#898989", linestyle="--", linewidth=1)
plt.legend(title="", ncol=2)

for ext in ["pdf", "png"]:
    plt.savefig(f"../../figures/figure-msv-cf.{ext}", bbox_inches="tight", dpi=300)